<a href="https://colab.research.google.com/github/jc890/python/blob/master/Movinet_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
MoViNet Model Module
-------------------
Responsibilities:
- Load pretrained MoViNet model from TensorFlow Hub
- Run inference on short video clips
- Return predicted action label ID

This module isolates all deep learning logic related to video understanding.
"""

import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

# =========================
# CONFIGURATION
# =========================
MOVINET_MODEL_URL = (
    "https://www.kaggle.com/models/google/movinet/"
    "TensorFlow2/a0-base-kinetics-600-classification/3"
)

# =========================
# LOAD MOVINET MODEL
# =========================
def load_movinet():
    """
    Loads the pretrained MoViNet model from TensorFlow Hub.

    IMPORTANT:
    - MoViNet must be called directly on real tensors
    - Do NOT wrap inside tf.keras.Model

    Returns:
        hub.KerasLayer
    """
    return hub.KerasLayer(
        MOVINET_MODEL_URL,
        trainable=False
    )

# =========================
# RUN MOVINET INFERENCE
# =========================
def run_movinet(video_frames):
    """
    Runs MoViNet inference on extracted video frames.

    Args:
        video_frames (np.ndarray):
            Shape (T, H, W, 3)
            Frames extracted using OpenCV

    Returns:
        int: Predicted action label ID (Kinetics-600)
    """

    if video_frames is None or len(video_frames) == 0:
        raise ValueError("Empty video frames provided to MoViNet.")

    # Normalize frames
    video_frames = video_frames.astype("float32") / 255.0

    # Add batch dimension
    video_tensor = tf.expand_dims(video_frames, axis=0)
    # Shape: (1, T, H, W, 3)

    movinet = load_movinet()

    # MoViNet expects dict input
    outputs = movinet(
        {"image": video_tensor},
        training=False
    )

    # Convert logits to probabilities
    probs = tf.nn.softmax(outputs, axis=-1)

    # Get predicted class index
    predicted_label = int(tf.argmax(probs, axis=-1).numpy()[0])

    return predicted_label


# =========================
# OPTIONAL TEST BLOCK
# =========================
if __name__ == "__main__":
    print("Running MoViNet model self-test...")

    # Dummy random video: 16 frames
    dummy_video = np.random.rand(16, 172, 172, 3).astype("float32")

    label_id = run_movinet(dummy_video)
    print(f"Test successful. Predicted label ID: {label_id}")

Running MoViNet model self-test...
Test successful. Predicted label ID: 129
